Primero cargamos todas las librerias que usaremos,en conjunto con los PATHS de las bases

In [3]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.stats.stattools import durbin_watson
from statsmodels.stats.diagnostic import acorr_breusch_godfrey
import statsmodels.api as sm
from statsmodels.compat import lzip
import datetime
from statsmodels.tools.tools import add_constant
import math
from scipy.stats import norm
from scipy.stats import chi2
#Cambiar el path *Importante
PATH_2210 = "E01_T_DEU_CONS_2210_REV.csv"
PATH_2211 = "E01_T_DEU_CONS_2211_REV.csv"
PATH_2212 = "E01_T_DEU_CONS_2212_REV.csv"
PATH_2301 = "E01_T_DEU_CONS_2301_REV.csv"
PATH_2302 = "E01_T_DEU_CONS_2302_REV.csv"
PATH_2303 = "E01_T_DEU_CONS_2303_REV.csv"
Lista_Path = [PATH_2210,PATH_2211,PATH_2212,PATH_2301,PATH_2302,PATH_2303]

In [4]:
def RutComunes(lista):
    dataframelist = []
    for i in lista:
        df = pd.read_csv(i, delimiter=";", usecols = ['RUT'])
        dataframelist.append(df)

    interseccion = np.array(dataframelist[0]['RUT'])

    for df in dataframelist[1:]:
        RutComunes = np.intersect1d(interseccion, df['RUT'])
        
    dfRutComunes = pd.DataFrame({'RUT': RutComunes})
    lista_rut = dfRutComunes["RUT"]
    return lista_rut

RutComunes(Lista_Path)

0                0
1                1
2                3
3                4
4                6
            ...   
1028140    1083121
1028141    1083122
1028142    1083123
1028143    1083124
1028144    1083125
Name: RUT, Length: 1028145, dtype: int64

Generamos el Test de Durbin Watson y el de tamaño muestra

In [8]:
def DataProm(dataframe):
    cantidad_filas = dataframe.shape[0]
    tamaño_muestra =calcular_tamano_muestra(cantidad_filas,0.05,0.5,0.05)
    lista_dataframes =[]
    for i in range (1000):
        numero_aleatorio = random.randint(tamano_muestra,cantidad_filas-tamaño_muestra)
        dfparalista = dataframe.iloc[numero_aleatorio-1:numero_aleatorio+tamaño_muestra]
        dfparalista.columns = dataframe.columns
        lista_dataframes.append(dfparalista)
    
    #print(lista_dataframes[30])
    num_dataframes = len(lista_dataframes)
    num_filas = lista_dataframes[0].shape[0]
    num_columnas = lista_dataframes[0].shape[1]
    df_promedio = pd.DataFrame(np.zeros((num_filas, num_columnas)))
    for i in range(num_filas):
        for j in range(num_columnas):
            suma = sum(df.iloc[i, j] for df in lista_dataframes)
            promedio = suma / num_dataframes
            df_promedio.iloc[i, j] = promedio
    df_promedio = df_promedio.drop(columns=df_promedio.columns[-1])
    a = 1
    n_columnas = df_promedio.shape[1]

    nuevos_nombres = {i: str(a + i) for i in range(n_columnas)}
    df_promedio = df_promedio.rename(columns=nuevos_nombres)
    return df_promedio



def TestSeriesDW (dataframe,fecha_1,fecha_2):

    df_fecha_1 = dataframe[str(fecha_1)]
    df_fecha_2 = dataframe[str(fecha_2)]
    dw_test = durbin_watson(df_fecha_2 - df_fecha_1)
    print(f"El test de DW entre las fechas de {str(fecha_1)} y {str(fecha_2)} es de :{dw_test}")


def calcular_tamano_muestra(N, alpha, p, d):
    z = abs(norm.ppf(1 - alpha/2)) 
    numerator = (N * z**2 * p * (1 - p))
    denominator = (d**2 * (N - 1) + z**2 * p * (1 - p))
    n = numerator / denominator
    muestra_tamaño = math.ceil(n)
    return muestra_tamaño
N = 1000000
alpha = 0.05
p = 0.5
d = 0.05

tamano_muestra = calcular_tamano_muestra(N, alpha, p, d)
print("Tamaño de la muestra:", tamano_muestra)

Tamaño de la muestra: 384


In [6]:
def Test(Variable,lista):
    lista_rut=RutComunes(Lista_Path)
    ###########################################################################
    dataframelist = []
    dataframelistna = []
    for i in lista:
        df1 = pd.read_csv(i, delimiter=";", usecols = ['RUT',Variable])
        df2 = df1.loc[df1['RUT'].isin(lista_rut)].dropna()
        dataframelist.append(df1)
        dataframelistna.append(df2)
    
    
    dataframes=[]
    a=1
    for i in dataframelistna:
        tupla = (i,str(a))
        dataframes.append(tupla)
        a += 1
    dftemporal = pd.DataFrame()
   
    
    for df, nombre_df in dataframes:
        dftemporal[nombre_df] = df[Variable]

    dftemporal['RUT'] = dataframelist[0]['RUT'] 
    dftemporal = dftemporal.dropna()

    
    dataframeDW = DataProm(dftemporal)
    TestSeriesDW(dataframeDW,1,2)
    TestSeriesDW(dataframeDW,5,6)
    TestSeriesDW(dataframeDW,6,7)#

Finalmente se llega a calcular Dw segun las fechas

In [9]:
Test("MTOINT",Lista_Path)

El test de DW entre las fechas de 1 y 2 es de :1.4558679503995113
El test de DW entre las fechas de 5 y 6 es de :1.5355026043078144


KeyError: '7'

In [38]:
def Test2(Variable):
    lista_rut=RutComunes(Lista_Path)
    df22101 = pd.read_csv(PATH_2210, delimiter=";", usecols = ['RUT',Variable])
    df22111 = pd.read_csv(PATH_2211, delimiter=";", usecols = ['RUT',Variable])
    df22121 = pd.read_csv(PATH_2212, delimiter=";", usecols = ['RUT',Variable])
    df23011 = pd.read_csv(PATH_2301, delimiter=";", usecols = ['RUT',Variable])
    df23021 = pd.read_csv(PATH_2302, delimiter=";", usecols = ['RUT',Variable])
    df23031 = pd.read_csv(PATH_2303, delimiter=";", usecols = ['RUT',Variable])
    dfvarible22101 = df22101.loc[df22101['RUT'].isin(lista_rut)].dropna()
    dfvarible22111 = df22111.loc[df22111['RUT'].isin(lista_rut)].dropna()
    dfvarible22121 = df22121.loc[df22121['RUT'].isin(lista_rut)].dropna()
    dfvarible23011 = df23011.loc[df23011['RUT'].isin(lista_rut)].dropna()
    dfvarible23021 = df22101.loc[df23021['RUT'].isin(lista_rut)].dropna()
    dfvarible23031 = df23031.loc[df23031['RUT'].isin(lista_rut)].dropna()

    dataframes = [
        (dfvarible22101, '1'),
        (dfvarible22111, '2'),
        (dfvarible22121, '3'),
        (dfvarible23011, '4'),
        (dfvarible23021, '5'),
        (dfvarible23031, '6')
    ]


    dftemporal = pd.DataFrame()

    for df, nombre_df in dataframes:
        
        dftemporal[nombre_df] = df[Variable]

    dftemporal['RUT'] = df22101['RUT'] 
    dftemporal = dftemporal.dropna()
    print(dftemporal)
    dataframeDW = DataProm(dftemporal)
    TestSeriesDW(dataframeDW,2210,2211)
    TestSeriesDW(dataframeDW,2211,2212)
    TestSeriesDW(dataframeDW,2212,2301)
    TestSeriesDW(dataframeDW,2301,2302)
    TestSeriesDW(dataframeDW,2302,2303)

In [39]:
Test2("MTOINT")

                 1         2         3       4          5        6     RUT
2              0.0   38805.0       0.0     0.0        0.0      0.0  897518
3           4494.0   40388.0    9854.0     0.0     4494.0      0.0  778606
4          12763.0   26671.0  136462.0     0.0    12763.0      0.0  778606
5           9621.0   27102.0  124570.0     0.0     9621.0      0.0  773271
6          16625.0   43084.0  106641.0     0.0    16625.0      0.0  318456
...            ...       ...       ...     ...        ...      ...     ...
2767498  8412768.0       9.0       0.0     0.0  8412768.0      0.0  579656
2767499        0.0   10948.0   75200.0     0.0        0.0      0.0  579655
2767500  9468364.0       0.0     228.0  7628.0  9468364.0  27551.0  579655
2767501  2469555.0  219514.0       0.0     0.0  2469555.0  10512.0  579663
2767502   638534.0  219514.0       0.0     0.0   638534.0      0.0  579661

[957525 rows x 7 columns]
El test de DW entre las fechas de 2210 y 2211 es de :1.821873278636483
El